In [3]:
# Ejemplo de importar csv desde GCS

# Actualizamos tf, keras y sklearn a la última versión
#!pip install --upgrade tensorflow
#!pip install --upgrade sklearn
# Si no, hacer uninstall e install: pip uninstall scikit-learn
#!pip install --upgrade keras
#!pip install h5py

import pandas as pd
from StringIO import StringIO
from sklearn.model_selection import train_test_split

# Read csv file from GCS into a variable
%storage read --object gs://analiticauniversal/DatasetsTF/creditcards.csv --variable creditcards

# Store in a pandas dataframe
df = pd.read_csv(StringIO(creditcards))
dataset = df.as_matrix()


X_train, X_test, y_train, y_test = train_test_split(dataset[:,:-1], dataset[:,-1], test_size=0.1, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [5]:
# -*- coding: utf-8 -*-

from __future__ import print_function
from __future__ import absolute_import

import numpy as np
import tensorflow as tf
import sys
import tensorflow.contrib.keras as keras


In [6]:
from keras.regularizers import l1,l2
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.constraints import max_norm
from keras.optimizers import RMSprop, Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score
from datetime import datetime
from os.path import abspath
import os


# Disable info warnings from TF
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

NOW = datetime.now().strftime("%Y-%m-%d--%Hh%Mm%Ss")
ROOT_LOGDIR = 'gs://examplesdata/PruebasTF'
LOG_DIR = abspath('{}/run-{}'.format(ROOT_LOGDIR, NOW))
CSV_LOG = abspath(LOG_DIR + '/training.log')
TXT_LOG = abspath(LOG_DIR + '/results.txt')
CKPT = abspath(LOG_DIR + '/ckpt.hdf5')
OUTPUT_FILE = abspath(TXT_LOG)
MODEL = abspath(LOG_DIR + '/model.h5')



Using TensorFlow backend.


In [45]:
# ENTENDER PATH BUCKETS Y SUBCARPETAS
# Ejemplos para como guardar archivos en buckets
from datalab.context import Context
import datalab.storage as storage

sample_bucket_name = Context.default().project_id + 
sample_bucket_path = 'gs://' + sample_bucket_name

sample_bucket = storage.Bucket(sample_bucket_name)

# Create storage bucket if it does not exist
if not sample_bucket.exists():
    sample_bucket.create()

# Write an item to the storage bucket
sample_item = sample_bucket.item('stringtofile.txt')
sample_item.write_to('This is a string', 'text/plain')

# Another way to copy an item from Datalab to Storage Bucket
#!gsutil cp 'someotherfile.txt' sample_bucket_path

RequestException: HTTP request failed: Invalid bucket name: 'analiticauniversal//DatasetsTF//'

In [39]:
from datalab.context import Context
import datalab.storage as storage

sample_bucket_name = Context.default().project_id
sample_bucket_path = 'gs://' + sample_bucket_name
sample_bucket = storage.Bucket(sample_bucket_name)

# Create storage bucket if it does not exist
if not sample_bucket.exists():
    sample_bucket.create()

In [29]:
# Hyperparameters
batch_size = 500
epochs = 200
dropout_rate = 0.5

In [30]:


x_train = X_train
y_train = y_train
x_val = X_val
y_val = y_val
x_test = X_test
y_test = y_test

input_dim = dataset.shape[1] - 1
num_classes = 2

model = Sequential()
model.add(Dense(5,input_shape=(input_dim,), init='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(dropout_rate))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1], "\n")



y_pred = model.predict_proba(x_test, verbose = 0)
y_score = y_pred[:,1]
auc = roc_auc_score(y_true=y_test, y_score=y_score)
auc *=100
print("Test AUC:", auc)




/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(5, kernel_initializer="he_normal", input_shape=(30,))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 5)                 155       
_________________________________________________________________
batch_normalization_9 (Batch (None, 5)                 20        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 12        
Total params: 187
Trainable params: 177
Non-trainable params: 10
_________________________________________________________________
Train on 230692 samples, validate on 25633 samples
Epoch 1/200
230692/230692 [==============================] - 1s - loss: 0.4139 - acc: 0.8221